In [2]:
# playing with data from https://www.kaggle.com/c/santander-customer-satisfaction

# trying convnets with Keras


import pandas as pd
import numpy as np
from time import gmtime, strftime

import tensorflow as tf
import tensorflow.contrib.learn as skflow
from sklearn.cross_validation import train_test_split

from sklearn import datasets, cross_validation, metrics, preprocessing
from sklearn.preprocessing import StandardScaler

print(strftime("%H:%M:%S", gmtime()))

data = pd.read_csv('train.csv')

data_id = data.pop('ID') # don't need

data = data.sample(frac=1) # shuffle rows /// .reset_index(drop=True)

data_target = data.pop('TARGET')

X, y = data, data_target
X = StandardScaler().fit_transform(X)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)

# adabas supports DecisionTreeClassifier or SGDClassifier

print(data.shape)
print(data_target.shape)

X, y = data, data_target

# scale values
# X = StandardScaler().fit_transform(X)
X.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)

print("data ready")


09:26:21
(76020, 369)
(76020,)
data ready


In [24]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, MaxPooling1D, Convolution1D
from keras.optimizers import SGD
from keras import backend as K


print("data.shape = ",data.shape)
# set parameters:
max_features = 5000
maxlen = 369
batch_size = 512
embedding_dims = 369
nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 2

# print('Loading data...')
# (X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features,
#                                                       test_split=0.2)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
# model.add(Embedding(max_features,
#                     embedding_dims,
#                     input_length=maxlen,
#                     dropout=0.2))

# model.add(Embedding(max_features, 128, input_length=maxlen, dropout=0.2))


# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(input_shape=X_train.shape,
                        nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))

print("model.inputs",model.inputs)
print("model.outputs",model.outputs)
# we use max over time pooling by defining a python function to use
# in a Lambda layer
def max_1d(X):
    return K.max(X, axis=1)

#model.add(Lambda(max_1d, output_shape=(nb_filter,)))

# We add a vanilla hidden layer:
#model.add(Dense(hidden_dims))
#model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

print("X_train.shape = ",X_train.shape)
print("y_train.shape = ",y_train.shape)
print("X_test.shape = ",X_test.shape)
print("y_test.shape = ",y_test.shape)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch)

# ,
#           validation_data=(X_test, y_test)


data.shape =  (76020, 369)
45612 train sequences
30408 test sequences
X_train shape: (45612, 369)
X_test shape: (30408, 369)
Build model...
model.inputs.shape [convolution1d_input_8]


Exception: Input 0 is incompatible with layer dense_19: expected ndim=2, found ndim=3